In [2]:
# 导入模块
import pandas as pd
import os
os.chdir('..')
# 将数据读入文件

file_path = 'C:\\Users\\Asus\\nmf\\'
file_name = 'topic_cleaned_data.csv'

papers = pd.read_csv(file_path + file_name,nrows = 6000)
papers = papers.drop(columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','Title','Abstract','Source','PubDate','Journal','DOI','FirstAuthor','OtherAuthors','Keywords','Languages'],axis=1)

# 输出显示前5个数据
papers.head()

,TextBody
0,vp vp processed vp vp virus assembly p p regio...
1,december novel coronavirus sarscov identified ...
2,outbreak infectious disease always one importa...
3,metagenomic sequencing allows u directly obtai...
4,since emergence december novel coronavirus sar...


In [3]:

papers= papers['TextBody']

papers = papers.sample(5000)

# 打印第一行
papers.head()

1691    december world health organization china count...
4944    variety virus large growing picornaviridae fam...
5173    member picornaviridae family positivestrand vi...
5551    introduction respiratory infection significant...
3148    hepatitis c virus hcv affect million people wo...
Name: TextBody, dtype: object

In [4]:
import gensim
from gensim.utils import simple_preprocess
# 加载正则表达式库
import re

# 删除标点符号
papers['paper_text_processed'] = papers.map(lambda x: re.sub('[,\.!?]', '', x))

#将标题转换为小写
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: x.lower())

# 打印第一排
papers['paper_text_processed'].head()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True 删除标点符号
data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['december', 'world', 'health', 'organization', 'china', 'country', 'office', 'informed', 'case', 'pneumonia', 'unknown', 'etiology', 'unknown', 'cause', 'detected', 'wuhan', 'city', 'hubei', 'province', 'china', 'reported', 'novel', 'coronavirus', 'ncov', 'named', 'severe', 'acute', 'respiratory', 'syndrome', 'coronavirus']


In [5]:
# 建立二联体和三联体模型
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# 这是把一个句子打成三字母或两字母组合的更快方法
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [6]:
# NLTK 停用词
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

D:\python\Anaconda\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
D:\python\Anaconda\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/r

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

data_words_nostops = remove_stopwords(data_words)
# 三元组

data_words_bigrams = make_bigrams(data_words_nostops)

# 初始化空间'en'模型，只保留标记组件(为了效率)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# 只保留n, v ,adj,  adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])


In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# 创建语料库
texts = data_lemmatized
# 文档频率
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

In [ ]:
# 创捷LDA模型
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)



In [ ]:
from gensim.models import CoherenceModel
# 计算连贯性得分
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:

def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
#LDA#LDA#LDA#LDA#LDA#LDA#LDA#LDA#LDA#LDA#LDA#LDA#LDA#LDA#LDA#LDA#LDA#LDA
coherence_scores=[0.254,0.337,0.358,0.377,0.386,0.408,0.412,0.414,0.415,0.419]
# 结果展示
fig = plt.figure(figsize=(15, 7))

plt.plot(
    coherence_scores,
    linewidth=3,
    color='#000000'
)

plt.xlabel("主题数目", fontsize=14)
plt.ylabel("连贯性评分", fontsize=14)
plt.title('根据主题数的连贯性评分 - 最好的主题数: 10', fontsize=18)
plt.xticks(np.arange(1, max(topic_nums) + 1, 1), fontsize=12)


plt.yticks(fontsize=12)

file_name = 'c_score'

fig.savefig(
    file_path + file_name + '.png',
    dpi=fig.dpi,
    bbox_inches='tight'
)

plt.show()

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,   #导入待训练语料
                                           id2word=id2word,  #设置构建模型的字典，决定了词汇数量
                                           num_topics=10,  #需要提取的潜在主题数                                     
                                           random_state=100, #随机化设置
                                           chunksize=5000,  #文件块大小
                                           passes=10,    #训练轮次数
                                           alpha=0.01,   #决定文档主题狄利克雷先验分布的超参数
                                           eta=0.9)     #决定主题词汇狄利克雷先验分布的超参数

In [ ]:
from pprint import pprint


pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

import pickle 

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared